In [73]:
# 交通資訊
from time import sleep
from collections import defaultdict
from datetime import datetime, timedelta
from tqdm import tqdm
import random
import os
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook
import pygsheets


header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}


In [74]:
# 抓金鑰
gc = pygsheets.authorize(service_file='ccclub_api_key.json')
# 連接雲端 sheet
sht = gc.open_by_url(
'https://docs.google.com/spreadsheets/d/1t7U4pvptpD_cfTMD4tUwDeCffAopPQF2hJEt386J18A/'
)


In [75]:
#匯入Excel(去重複+排序)
def append_to_excel(new_data, sheet_name, pk):
    try:
        # 嘗試讀取現有的 Excel 檔案
        ws = sht.worksheet_by_title(sheet_name)
        existing_df = pd.DataFrame(ws.get_all_records())
    except Exception as e:
        # 如果檔案不存在，創建一個空的 DataFrame
        ws = sht.add_worksheet(sheet_name)
        existing_df = pd.DataFrame()
    # 合併現有的資料和新資料
    combined_df = pd.concat([existing_df, new_data], ignore_index=True)

    # 移除重複的 ID，保留最新的記錄
    combined_df.drop_duplicates(subset=[pk], keep='last', inplace=True)

    # 根据 pk 排序
    if pk in combined_df.columns:
        combined_df.sort_values(by=pk, inplace=True)
    # 將合併後的資料寫入 Excel 檔案
    ws.set_dataframe(combined_df, (1,1),nan='')


In [76]:
sheet_name = 'trail_information'
sheet_name2 = 'mountain_list'
pk = '步道ID'
pk2 = '山岳ID'

In [77]:
#建立步道ID index
allMid_url = "https://hiking.biji.co/index.php?q=trail"
idx_r = requests.get(allMid_url, headers=header)
idx_soup = BeautifulSoup(idx_r.text, 'html.parser')

#取得最大頁數
allPage_num = idx_soup.select('a[class="page-item grid place-content-center p-1.5 bg-white text-gray-600 border border-gray-300 rounded"]')
max_page = 0
for allPage in allPage_num:
    allPage_href = allPage.get('href')
    if "page=" in allPage_href:
        page_id = int(allPage_href.split('page=')[1])
        if page_id > max_page:
            max_page = page_id
print(max_page)

93


In [78]:
allMid_base_url = "https://hiking.biji.co/index.php?q=trail&page="
all_idx_id = []
error_page=[]
page=0
try:
    for p in tqdm(range(1, max_page+1)):
        sleep(random.uniform(1, 2))

        allMid_page_url = allMid_base_url + str(p)
        P_idx_r = requests.get(allMid_page_url, headers=header)
        P_idx_soup = BeautifulSoup(P_idx_r.text, 'html.parser')

        idx_contents_get = P_idx_soup.select('a[class="text-current"]')
        base_url = "https://hiking.biji.co/index.php?q=trail&act=detail&id="
        for idx_content in idx_contents_get:
            idx_Mname = idx_content.text
            idx_href = idx_content.get("href")
            if "id=" in idx_href:
                #獲得這一頁上的連結(ID)
                idx_id = int(idx_href.split('id=')[1])
                if idx_id not in all_idx_id:
                    all_idx_id.append(idx_id)

                sleep(random.uniform(1, 2))

                trail_id = idx_id
                url = base_url + str(trail_id)
                r = requests.get(url, headers=header)
                soup = BeautifulSoup(r.text, 'html.parser')

                trail_name={}
                trail_name['步道ID'] = trail_id


                trail_infos={}
                # 步道名稱
                title = soup.select_one('meta[property="og:title"]')['content']
                trail_infos['步道名稱'] = title
                double_fields_infos = soup.select('div[class="flex divide-x divide-gray-200"]')
                single_fields_infos = soup.select('div[class="flex"]')
                for info in double_fields_infos:
                    if info.select_one('dt').text == '所在縣市':
                        # 所在縣市、里程
                        city,distance = list(map(lambda x:x.text.replace('\n',''),info.select('dd')))
                        trail_infos['所在縣市'] = city
                        trail_infos['里程'] = distance
                    if info.select_one('dt').text == '步道類型':
                        # 步道類型、步道型態
                        trail_class,trail_type = list(map(lambda x:x.text.replace('\n',''),info.select('dd')))
                        trail_infos['步道類型'] = trail_class
                        trail_infos['步道型態'] = trail_type
                    if info.select_one('dt').text == '海拔高度':
                        # 海拔高度、高度落差
                        height,height_difference =list(map(lambda x:x.text.replace('\n',''),info.select('dd')))
                        trail_infos['海拔高度'] = height
                        trail_infos['高度落差'] = height_difference
                    if info.select_one('dt').text == '路面狀況':
                        # 路面狀況、所需時間
                        road,time =list(map(lambda x:x.text.replace('\n',''),info.select('dd')))
                        trail_infos['路面狀況'] = road
                        trail_infos['所需時間'] = time
                    if info.select_one('dt').text == '難易度':
                        # 難易度、所屬園區
                        difficulty,belong_park = list(map(lambda x:x.text.replace('\n',''),info.select('dd')))
                        trail_infos['難易度'] = difficulty
                        trail_infos['所屬園區'] = belong_park
                    if info.select_one('dt').text == '山脈':
                        # 山脈、最適季節
                        belong_mountain,season = list(map(lambda x:x.text.replace('\n',''),info.select('dd')))
                        trail_infos['山脈'] = belong_mountain
                        trail_infos['最適季節'] = season
                    if info.select_one('dt').text == '申請入山':
                        # 申請入山、申請入園
                        apply_mountain,apply_park = list(map(lambda x:x.text.replace('\n',''),info.select('dd')))
                        trail_infos['申請入山'] = apply_mountain
                        trail_infos['申請入園'] = apply_park

                mountain_dict={}
                for info in single_fields_infos:
                    if '百岳' in info.select_one('dt').text:
                        # 百岳/小百岳
                        peaks = list(map(lambda x:x.text.replace('\n',''),info.select('li')))
                        trail_infos['百岳/小百岳'] = '、'.join(peaks)
                    if info.select_one('dt').text == '附近步道':
                        # 附近步道
                        nearby_trails = list(map(lambda x:x.text.replace('\n',''),info.select('li')))
                        trail_infos['附近步道'] = '、'.join(nearby_trails)
                    if info.select_one('dt').text == '相關山岳':
                        # 相關山岳
                        names=[]
                        ids=[]
                        for i in info.select('li'):
                            target=i.select("a")[0].get('href')
                            ind=target.index("&id")
                            id = target[ind+4:]
                            mountain_dict['山岳ID']=int(id)
                            mountain_dict['山岳名稱']=i.text.replace('\n','')
                            names.append(i.text.replace('\n',''))
                            ids.append(id)
                        trail_infos['相關山岳名稱'] = '、'.join(names)
                        trail_infos['相關山岳ID'] = ','.join(ids)
                    if info.select_one('dt').text == '相關植物':
                        # 相關植物
                        plants = list(map(lambda x:x.text.replace('\n',''),info.select('li')))
                        trail_infos['相關植物'] = '、'.join(plants)
                    if info.select_one('dt').text == '基點':
                        # 基點
                        reference_point = list(map(lambda x:x.text.replace('\n',''),info.select('dd')))
                        trail_infos['基點'] = '\n'.join(reference_point)

                category_traffic = soup.select('section[id="category_traffic"]')[0]
                transport_infos = defaultdict(list)
                categories = category_traffic.select('div[class="flex"]')
                for category in categories:
                    category_name= category.select_one('dt').text
                    if category_name=='路線規劃':
                        break
                    ways = category.select('dd')
                    for way in ways:
                        newWays = way.text.split("【")
                        for newWay in newWays:
                            try:
                                ind2=newWay.index("】")
                                name = newWay[:ind2]
                                transport_infos[f'交通-{category_name}'].append(f'{name}:{newWay[ind2+1:]}')
                            except:
                                continue

                for key in transport_infos.keys():
                    transport_infos[key] = '\n'.join(transport_infos[key])
                merged_dict = {**trail_name, **trail_infos, **transport_infos}
                trail_data = []
                trail_data.append(merged_dict)
                trail_df = pd.DataFrame(trail_data)

                #放入pandas DataFrame(山岳基本資料)
                mountain_data = []
                mountain_data.append(mountain_dict)
                mountain_df = pd.DataFrame(mountain_data)
                #寫入Excel
                append_to_excel(trail_df, sheet_name, pk)
                append_to_excel(mountain_df, sheet_name2, pk2)
        page=p
except:
    error_page.append(str(page))
print(','.join(error_page))
print(all_idx_id)
print(len(all_idx_id))

100%|██████████| 93/93 [3:03:50<00:00, 118.61s/it]  


[76, 54, 171, 71, 108, 228, 30, 759, 288, 87, 413, 303, 287, 525, 490, 289, 400, 314, 41, 74, 34, 86, 264, 194, 55, 355, 69, 70, 381, 446, 75, 105, 249, 37, 370, 315, 299, 88, 309, 247, 387, 561, 373, 382, 492, 56, 232, 290, 404, 491, 165, 660, 445, 73, 10, 170, 89, 131, 562, 552, 286, 47, 49, 234, 201, 664, 140, 258, 50, 357, 399, 9, 593, 333, 431, 130, 265, 486, 435, 310, 94, 35, 862, 78, 748, 270, 766, 715, 169, 520, 191, 430, 425, 125, 263, 327, 347, 36, 389, 212, 40, 57, 524, 48, 160, 308, 325, 428, 686, 793, 113, 104, 115, 250, 463, 410, 368, 183, 139, 418, 329, 53, 192, 266, 241, 330, 262, 401, 738, 267, 259, 124, 437, 61, 260, 6, 338, 485, 1018, 268, 536, 271, 572, 851, 256, 243, 173, 848, 461, 121, 65, 537, 402, 385, 559, 238, 340, 427, 185, 42, 297, 448, 1160, 51, 494, 155, 135, 72, 480, 913, 779, 1169, 336, 602, 298, 316, 64, 235, 60, 58, 44, 813, 326, 456, 1204, 580, 106, 380, 776, 230, 473, 269, 248, 321, 487, 597, 672, 5, 534, 444, 596, 592, 372, 38, 184, 1276, 504, 823,